In [1]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score ,GridSearchCV
import joblib
import os

In [ ]:
# df = pd.read_csv(f"./train_combine_groupby/01_traingroupby.csv")
# df = addcolumns(df)
# def combine_weather_10mins(df1:pd.DataFrame):
#     weather_df = pd.read_csv("../資料前處理以及第三方資料/10mins_weather_data.csv")
#     weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
#     weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
#     weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
#     weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
#     weather_df.fillna(0,inplace=True)
#     df = pd.merge(
#         df1,
#         weather_df,
#         left_on=["日期","時間","時段"],
#         right_on=["日期","時間","時段"],
#         how="left",
#         suffixes=["","_r"]
#     )

#     return(df)
# df = combine_weather_10mins(df)

In [ ]:
# def combine_weather_1hour(df1:pd.DataFrame):
#     weather_df = pd.read_csv("../資料前處理以及第三方資料/station466990weather.csv")
#     weather_df = weather_df.loc[:,["DateTime","TX01","GR01","RH01","UV01"]]
#     weather_df["DateTime"] = weather_df["DateTime"].astype(str)
#     weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].str.slice(0,8))
#     weather_df["時間"] = weather_df["DateTime"].str.slice(8,10)
    
#     weather_df.fillna(0,inplace=True)
#     df = pd.merge(
#         df1,
#         weather_df,
#         left_on=["日期","時間"],
#         right_on=["日期","時間"],
#         how="left",
#         suffixes=["","_r"]
#     )

#     return(df)
# df = combine_weather_1hour(df)

In [ ]:
# def combine_sundf(df:pd.DataFrame):
#     sun_df = pd.read_csv("../太陽仰角與日射量資料計算整理/sunlight_data.csv")
#     sun_df["location"] = sun_df["location"].astype(str).str.zfill(2)
#     sun_df["時間"] = pd.to_datetime(sun_df["時間"])
#     ["太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"]
#     df = df.merge(
#         sun_df.loc[:,["location","時間","太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"]],
#         left_on=["TimeGroup","LocationCode"],
#         right_on=["時間","location"],
#         suffixes=["","_r"],
#         how="left"
#     )
#     return(df)
# 0

In [12]:
def selecttime(df:pd.DataFrame):
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df["LocationCode"] = df["LocationCode"].astype(str).str.zfill(2)
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)

In [11]:
weather_df = pd.read_csv(f"../資料前處理以及第三方資料/weather_sun_combine.csv")
weather_df["日期"] = pd.to_datetime(weather_df["日期"])
weather_df["location"] = weather_df["location"].astype(str).str.zfill(2)
weather_df["時間"] = weather_df["時間"].astype(str).str.zfill(2)
weather_df["時段"] = weather_df["時段"].astype(str).str.zfill(2)
weather_df.head(2)

,location,DateTime,太陽仰角,日射量 (W/m²),太陽方位角,太陽能板日射量 (W/m²),StationId,SunshineDuration,UVIndex,AirTemperature,...,StationLongitude,StationAltitude,日期,時間,時段,DateTime_r,TX01,GR01,RH01,UV01
0,01,2024-01-01 00:00:00,-88.366208,0.0,138.515569,0.0,466990.0,0.9,0.0,19.2,...,121.61327,16.1,2024-01-01,00,00,2024010124,17.1,0.0,79,0.0
1,01,2024-01-01 00:10:00,-86.202505,0.0,137.831949,0.0,466990.0,0.0,0.0,19.2,...,121.61327,16.1,2024-01-01,00,10,2024010124,17.1,0.0,79,0.0


In [14]:
d = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"./train_combine_groupby/{location}_traingroupby.csv")
    
    df = addcolumns(df)
    # df = combine_weather_10mins(df)
    # df = combine_weather_1hour(df)
    df = selecttime(df)
    # df = combine_sundf(df)
    df = df.merge(
        weather_df.loc[weather_df["location"]==location,],
        left_on=["日期","時間",	"時段"],
        right_on=["日期","時間","時段"],
    )
    select_cols = [ "TX01","GR01","RH01","UV01",
                    "AirTemperature","RelativeHumidity","SunshineDuration","UVIndex",
                    "太陽仰角", "日射量 (W/m²)","太陽方位角", "太陽能板日射量 (W/m²)"
                    ]
    
    X = df.loc[:,select_cols].reset_index(drop=True)
    Y = df.loc[:,["Power(mW)"]].reset_index(drop=True)
    Y = Y.values.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

    model = RandomForestRegressor(n_jobs=14)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
    }
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=5,
        n_jobs=14,
    )
    grid_search.fit(X_train, y_train)
      # 儲存最佳模型和 MSE
    best_model = grid_search.best_estimator_
    scores = cross_val_score(best_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    MAE = -sum(scores)
    best_model.fit(X_train, y_train)
    
    model_file_name = f'{location}_weather.pkl'  
    model_folder_name = "repeat_train_weatherdata_all"         #將模型相關資訊儲存在這個檔案下
    model_file_path = os.path.join("./model_each_location",model_folder_name,model_file_name)
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    joblib.dump(best_model, model_file_path)
    print(model_file_name ," MAE:",MAE )
    print(f"模型已保存至: {model_file_path}")
    if model_file_name not in d:
        d[location] = MAE
    

01_weather.pkl  MAE: 540.7224538112295
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\01_weather.pkl
02_weather.pkl  MAE: 606.0355887231256
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\02_weather.pkl
03_weather.pkl  MAE: 702.1784764139625
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\03_weather.pkl
04_weather.pkl  MAE: 555.6503402813032
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\04_weather.pkl
05_weather.pkl  MAE: 715.2152888187578
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\05_weather.pkl
06_weather.pkl  MAE: 777.1923232991273
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\06_weather.pkl
07_weather.pkl  MAE: 674.2115181761526
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\07_weather.pkl
08_weather.pkl  MAE: 447.3072676581093
模型已保存至: ./model_each_location\repeat_train_weatherdata_all\08_weather.pkl
09_weather.pkl  MAE: 343.9548945879543
模型已保存至: ./model_each_location\repeat_train_weatherdata_al

In [15]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"./model_each_location/{model_folder_name}/d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)